## Import libraries

In [ ]:
%%time
import os
import pandas as pd
import numpy as np
from random import randint

## Import df

In [ ]:
%%time
def load_df():
    
    global df_path
    global df
    
    df_path = input("Please insert the CSV file path and type Enter >>>")

    def path_check(path):
        if os.path.exists(path):
            accept_ext = [".csv"]
            split_path = os.path.splitext(path)
            if split_path[1] not in accept_ext:
                print("Invalid file format. Please insert the path to a CSV file: ")
                return False
            else:
                return True
        else:
            print("Invalid path. Please insert a valid path to a CSV file: ")
            return False

    while not path_check(df_path):
        df_path = input()

    print("Valid path.")
    
    df = pd.read_csv(df_path, index_col=0, sep=';')
    df['int'],df['int_horizon'],df['bending_prob'],df['bending'],df['vibrato'],df['ho_prob'],df['hammer_on'],df['po_prob'],df['pull_off'],df['slide_legato'],df['pick'] = [0,0,0,0,0,0,0,0,0,0,0]
    print(df.head(30))

load_df()

## Bending

In [ ]:
%%time

# Calculate valid pitches

print("Please type the reference scale in numbers (intervals), starting from C. \nFor example C major scale is 0,2,4,5,7,9,11.\nPress the Enter button after each value, and type \"END\" after the last one.")
escape = False
valid_notes = np.array([])
while escape == False:
    new_value = input("Type next value and press the Enter button")
    if new_value != "END":
        new_value = new_value
        valid_notes = np.append(valid_notes, new_value)
    else:
        break

valid_notes = valid_notes.astype('int')        

len_scale = -len(valid_notes)
for i in range(8):
    valid_notes = np.append(valid_notes, valid_notes[len_scale:] + 12)
    
print(type(valid_notes[0]))
print(valid_notes)

# Set bending general parameters    

custom_parameters_bending = input("Do you want to customize the expressive model parameters regarding bending? Type Y or N")
if custom_parameters_bending != "N":
    custom_parameters_bending = True
    bending_min_duration_ms = int(input("Type the note min duration in ms to apply a bending"))
    bending_stats_deviation =  input("Do you want to modify the target stats for the application of bending? Type Y or N")
    if bending_stats_deviation == "Y":
        bending_stats_deviation_perc = int(input("Type the deviation desired, in percentage"))
else:
    custom_parameters_bending = False

if not custom_parameters_bending:
    bending_min_duration_ms = 120 # min note duration in ms to apply a bending
    bending_stats_deviation_perc = 0
    
max_bending = [2, 3, 4, 0] # max bending for each finger 1st to 4th 

bending_frequency_dataset = np.array([6.7,7.7,8.7,0.9,1.4,1.1]) # Bending frequency for each string in percentage - 1st to 6th

# Initiate the array with the number of notes on each string, first to sixth
number_of_notes_each_string = np.array([0,0,0,0,0,0])

for i in df.index:
    number_of_notes_each_string[df.at[i, 'string']-1] += 1
    
# Calculate the target number of bendings for each string

target_number_bends = (number_of_notes_each_string * (bending_frequency_dataset + (bending_frequency_dataset * bending_stats_deviation_perc / 100)) / 100).astype(int)

print("Number of notes for each string, 1st to 6th: ", number_of_notes_each_string)
print("Target number of bends for each string, 1st to 6th: ", target_number_bends)

actual_bends_each_string = np.array([0,0,0,0,0,0])

# Generate intervals direction labels

for i in range(len(df.index) -1):
    if df.at[i, 'note_numb'] - df.at[i+1, 'note_numb'] < 0:
        df.at[i, 'int'] = "asc"
        df.at[i, 'int_horizon'] = "asc"
    elif  df.at[i, 'note_numb'] - df.at[i+1, 'note_numb'] > 0:
        df.at[i, 'int'] = "desc"
        df.at[i, 'int_horizon'] = "desc"
    elif  df.at[i, 'note_numb'] - df.at[i+1, 'note_numb'] == 0:
        df.at[i, 'int'] = "unison"
        df.at[i, 'int_horizon'] = float("nan")
        
df.at[len(df.index) -1, 'int'] = "None"
df.at[len(df.index) -1, 'int_horizon'] = "None"

df['int_horizon'].bfill(inplace=True)

# Find nearest lower note and returns the distance from the actual one

def find_nearest_lower_note_int(note_numb):
    valid_lower_notes = valid_notes[(note_numb > valid_notes)]
    nearest_lower_note = np. max(valid_lower_notes)
    bend_start_interval = note_numb - nearest_lower_note
    return(bend_start_interval)


# Check there are not other bendings
    
def no_current_prev_bending():
    if df.at[i-1, 'bending'] == 0 or df.at[i, 'bending'] == 0:
        return True
    else:
        return False

def no_current_fllw_bending():
    if df.at[i, 'bending']== 0 or df.at[i+1, 'bending'] == 0:
        return True
    else:
        return False
    
def no_current_prev_fllw_bending():
    if df.at[i-1, 'bending'] == 0 or df.at[i, 'bending'] == 0 or df.at[i+1, 'bending'] == 0:
        return True
    else:
        return False    

# Bending on lower string generation

def prev_finger_correction():
    if i == 0:
        pass
    else:
        if df.at[i, 'string'] == df.at[i-1, 'string'] and df.at[i, 'fret'] == df.at[i-1, 'fret']:
            df.at[i-1, 'finger'] = df.at[i, 'finger']

def bend_lower_string_generation():
    if df.at[i, 'string'] == 2: # Bending on 3rd string                  
        if bend_start_interval <= 2: # Less or equal to 2 half steps --> bending with the ring finger
            if max_bending[2] >= bend_start_interval:
                def bend_func_3rd_str_3rd_finger():
                    if target_number_bends[df.at[i, 'string']] - actual_bends_each_string[df.at[i, 'string']] >= 0:
                        df.at[i, 'string'] = df.at[i, 'string'] + 1
                        df.at[i, 'fret'] = df.at[i, 'fret'] + (4 - bend_start_interval)
                        df.at[i, 'finger'] = 3
                        df.at[i, 'bending'] = bend_start_interval
                        actual_bends_each_string[df.at[i, 'string']-1] += 1
                        prev_finger_correction()
                if i == 0:
                    bend_func_3rd_str_3rd_finger()
                elif df.at[i-1, 'finger'] != 3:
                    bend_func_3rd_str_3rd_finger()
                else:
                    pass
        if bend_start_interval == 3: # Equal to 3 half steps --> bending with the middle finger
            if max_bending[1] >= bend_start_interval:
                def bend_3rd_str_2nd_finger():
                    if target_number_bends[df.at[i, 'string']] - actual_bends_each_string[df.at[i, 'string']] >= 0:
                        df.at[i, 'string'] = df.at[i, 'string'] + 1
                        df.at[i, 'fret'] = df.at[i, 'fret'] + (4 - bend_start_interval)
                        df.at[i, 'finger'] = 2
                        df.at[i, 'bending'] = bend_start_interval
                        actual_bends_each_string[df.at[i, 'string']-1] += 1
                        prev_finger_correction()
                if i == 0:
                    bend_func_3rd_str_2nd_finger()
                elif df.at[i-1, 'finger'] != 2:
                    bend_func_3rd_str_2nd_finger()
                else:
                    pass
    else: # Bending on other strings (not 3rd)
        if bend_start_interval <= 3: # Less or equal to 3 half steps --> bending with the ring finger
            if max_bending[2] >= bend_start_interval:
                def bend_func_no3rd_str_3rd_finger():
                    if target_number_bends[df.at[i, 'string']] - actual_bends_each_string[df.at[i, 'string']] >= 0:
                        df.at[i, 'string'] = df.at[i, 'string'] + 1
                        df.at[i, 'fret'] = df.at[i, 'fret'] + (5 - bend_start_interval)
                        df.at[i, 'finger'] = 3
                        df.at[i, 'bending'] = bend_start_interval
                        actual_bends_each_string[df.at[i, 'string']-1] += 1
                        prev_finger_correction()
                if i == 0:
                    bend_func_no3rd_str_3rd_finger()
                elif df.at[i-1, 'finger'] != 3:
                    bend_func_no3rd_str_3rd_finger()
                else:
                    pass
        if bend_start_interval == 4:  # Equal to 4 half steps --> bending with the middle finger
            if max_bending[1] >= bend_start_interval:
                def bend_no3rd_str_2nd_finger(): 
                    if target_number_bends[df.at[i, 'string']] - actual_bends_each_string[df.at[i, 'string']] >= 0:
                        df.at[i, 'string'] = df.at[i, 'string'] + 1
                        df.at[i, 'fret'] = df.at[i, 'fret'] + (5 - bend_start_interval)
                        df.at[i, 'finger'] = 2
                        df.at[i, 'bending'] = bend_start_interval
                        actual_bends_each_string[df.at[i, 'string']-1] += 1
                        prev_finger_correction()
                if i == 0:
                    bend_func_no3rd_str_2nd_finger()
                elif df.at[i-1, 'finger'] != 3:
                    bend_func_no3rd_str_2nd_finger()
                else:
                    pass 

# In case of unison interval and ascending horizon interval
                
def unis_asc_bend():
    if df.at[i, 'int'] == "unison" and df.at[i, 'int_horizon'] == "asc":
        if df.at[i, 'finger'] == 1 and df.at[i, 'string'] != 6 and df.at[i+1, 'finger'] == 1:
            bend_lower_string_generation()
        
# In case of unison interval and descending horizon interval
                        
def unis_desc_bend():
    if no_current_prev_fllw_bending() and df.at[i-1, 'int'] == "unison" and df.at[i-1, 'int_horizon'] == "desc":
        if df.at[i, 'finger'] == 1 and df.at[i, 'string'] != 6 and df.at[i-1, 'finger'] == 1:
            bend_lower_string_generation()
            if df.at[i, 'string'] == df.at[i+1, 'string']  and df.at[i, 'fret'] == df.at[i+1, 'fret']: # If following note is on the same string/fret use the same finger
                df.at[i+1, 'finger'] = df.at[i, 'finger']

# In case of groups of three notes with the first and third of the same pitch and the second higher (within max_bend)

def three_grp_up():
    bending = df.at[i+1, 'note_numb'] - df.at[i, 'note_numb']
    if no_current_fllw_bending() and df.at[i+2, 'bending'] == 0 and df.at[i, 'string'] == df.at[i+2, 'string'] and df.at[i, 'fret'] == df.at[i+2, 'fret'] and df.at[i, 'note_numb'] == df.at[i+2, 'note_numb'] and bending > 0 and bending <= max_bending[df.at[i, 'finger']-1]:
        if target_number_bends[df.at[i, 'string']-1] - actual_bends_each_string[df.at[i, 'string']-1] >= 0:
            df.at[i+1, 'string'] = df.at[i, 'string']
            df.at[i+1, 'fret'] = df.at[i, 'fret']
            df.at[i+1, 'finger'] = df.at[i, 'finger']
            df.at[i+1, 'bending'] = bending
            df.at[i+1, 'pick'] = False
            df.at[i+2, 'pick'] = False
            actual_bends_each_string[df.at[i, 'string']-1] += 1  
                      
# In case of small descending interval on same string (and same fret with bending)               

def small_int_desc():
    bend_interval = df.at[i, 'note_numb'] - df.at[i+1, 'note_numb']
    if bend_interval > max(max_bending) or bend_interval < 1 or not no_current_prev_fllw_bending():
        pass
    else:
        if df.at[i, 'string'] == df.at[i+1, 'string'] and df.at[i-1, 'fret'] != df.at[i+1, 'fret']:
            if bend_interval <= max_bending[df.at[i+1, 'finger']-1]:
                if target_number_bends[df.at[i, 'string']-1] - actual_bends_each_string[df.at[i, 'string']-1] >= 0:
                    df.at[i, 'fret'] = df.at[i+1, 'fret']
                    df.at[i, 'finger'] = df.at[i+1, 'finger']        
                    df.at[i, 'bending'] = bend_interval
                    actual_bends_each_string[df.at[i, 'string']-1] += 1
            else:
                pass 

# In case of small ascending interval on same string                

def small_int_asc():
    bend_interval = df.at[i+1, 'note_numb'] - df.at[i, 'note_numb']
    if i == len(df.index) - 2 or bend_interval > max(max_bending) or bend_interval < 1 or not no_current_prev_fllw_bending():
        pass
    else:
        if df.at[i, 'string'] == df.at[i+1, 'string']:
            if bend_interval <= max_bending[df.at[i, 'finger']] and df.at[i+2, 'finger'] != df.at[i, 'finger']:
                if target_number_bends[df.at[i, 'string']-1] - actual_bends_each_string[df.at[i, 'string']-1] >= 0:
                    df.at[i+1, 'fret'] = df.at[i, 'fret']
                    df.at[i+1, 'finger'] = df.at[i, 'finger']
                    df.at[i+1, 'bending'] = bend_interval
                    actual_bends_each_string[df.at[i, 'string']-1] += 1
            else:
                pass  

# In case of note followed by a rest or last note

def note_rest_end():
    if i == 0:
        if no_current_fllw_bending() and df.at[i, 'fllw_rest'] > 0:
            if df.at[i, 'finger'] == 1 and df.at[i, 'string'] != 6:
                bend_lower_string_generation()
                print("note_rest_end at ID " + str(i)) # <<<<---- Remove
            elif df.at[i, 'finger'] != 1:
                if target_number_bends[df.at[i, 'string']-1] - actual_bends_each_string[df.at[i, 'string']-1] >= 0:
                    df.at[i, 'fret'] = df.at[i, 'fret'] - bend_start_interval
                    df.at[i, 'finger'] = 3
                    df.at[i, 'bending'] = bend_start_interval
                    actual_bends_each_string[df.at[i, 'string']-1] += 1
            else:
                pass
    elif i == (len(df.index) - 1):
        if no_current_prev_bending() and (df.at[i, 'finger'] == 1 or df.at[i, 'prec_rest'] > 0):
            bend_lower_string_generation()
    else:
        if no_current_prev_fllw_bending() and df.at[i, 'fllw_rest'] > 0:
            if df.at[i, 'finger'] == 1:
                bend_lower_string_generation()
        
# Bending generation                 

for i in range(len(df.index)-1):
    if df.at[i, 'note_lng_ms'] < bending_min_duration_ms: # Check min duration
        pass
    else:
        bend_start_interval = find_nearest_lower_note_int(df.at[i, 'note_numb'])
        unis_asc_bend()
        
for i in range(1,len(df.index)-1):
    if df.at[i, 'note_lng_ms'] < bending_min_duration_ms: # Check min duration
        pass
    else:
        bend_start_interval = find_nearest_lower_note_int(df.at[i, 'note_numb'])
        unis_desc_bend()
        
for i in range(len(df.index)-2):
    if df.at[i, 'note_lng_ms'] < bending_min_duration_ms: # Check min duration
        pass
    else:
        three_grp_up()
        
for i in range(1,len(df.index)-1):
    if df.at[i, 'note_lng_ms'] < bending_min_duration_ms: # Check min duration
        pass
    else:
        small_int_desc()
        
for i in range(1,len(df.index)-1):
    if df.at[i, 'note_lng_ms'] < bending_min_duration_ms: # Check min duration
        pass
    else:
        small_int_asc()
        
for i in range(len(df.index)):
    if df.at[i, 'note_lng_ms'] < bending_min_duration_ms: # Check min duration
        pass
    else:
        bend_start_interval = find_nearest_lower_note_int(df.at[i, 'note_numb'])
        note_rest_end()
        
# Initiate again the array with the number of notes on each string, first to sixth (bends application may have changed the distribution)
number_of_notes_each_string = np.array([0,0,0,0,0,0])

for i in df.index:
    number_of_notes_each_string[df.at[i, 'string']-1] += 1

print("Actual bends on each string: ", actual_bends_each_string)

print(df[['note_name', 'int', 'int_horizon', 'string', 'fret', 'finger', 'note_lng_ms', 'bending', 'pick']].head(50))

df.to_csv('out_bending.csv')

## Vibrato

In [ ]:
%%time
custom_parameters_vibrato = input("Do you want to customize the expressive model parameters regarding vibrato? Type Y or N")

if custom_parameters_vibrato != "N":
    custom_parameters_vibrato = True
    vibrato_min_duration_ms = int(input("Type the note min duration in ms to apply a vibrato"))
    vibrato_hp_min_duration_ms = int(input("Type the note min duration  in ms to consider the vibrato highly probable"))
    always_vibrato_on_compatible_notes = input("Do you want to always apply vibrato to compatible notes? Type Y or N")
    if always_vibrato_on_compatible_notes == "N":
        always_vibrato_on_longer_notes = input("Do you want to always apply vibrato to longer notes? Type Y or N")
    else:
        always_vibrato_on_longer_notes = "Y"
    vibrato_stats_deviation =  input("Do you want to modify the target stats for the application of vibrato? Type Y or N")
    if vibrato_stats_deviation == "Y":
        vibrato_stats_deviation_perc = int(input("Type the deviation desired, in percentage"))
    
else:
    custom_parameters_vibrato = False

if not custom_parameters_vibrato:
    vibrato_min_duration_ms = 400 # Min ms note duration to allow a vibrato
    vibrato_hp_min_duration_ms = 800 # Min ms note duration to consider a vibrato highly probable
    always_vibrato_on_longer_notes = "Y"
    always_vibrato_on_compatible_notes = "N"

vibrato_frequency_dataset = [4.7, 5.2, 5.6, 4.4, 2.6, 2.0] # Probability of having a vibrato for each string, from the first to the sixth
vibrato_hp_options = np.array([0,0,0,0,0,0]) # Options to insert a high probability vibrato, for each string, first to sixth
vibrato_options = np.array([0,0,0,0,0,0]) # Options to insert vibrato, for each string, first to sixth
vibrato_hp_options_perc = np.array([0,0,0,0,0,0]) # Percentage options to insert a high probability vibrato, for each string, first to sixth
vibrato_options_perc = np.array([0,0,0,0,0,0]) # Percentage options to insert vibrato, for each string, first to sixth

# Identify notes with high probability of vibrato and those to which vibrato can be applied
for i in df.index:
    if df.at[i, 'fret'] != 0:
        if df.at[i, 'note_lng_ms'] >= vibrato_hp_min_duration_ms:   # not on open string and enough note duration
            df.at[i, 'vibrato_prob'] = 2
            vibrato_hp_options[df.at[i, 'string']-1] += 1
        elif df.at[i, 'note_lng_ms'] >= vibrato_min_duration_ms:   # not on open string and enough note duration
            df.at[i, 'vibrato_prob'] = 1
            vibrato_options[df.at[i, 'string']-1] += 1
    else:
        df.at[i, 'vibrato_prob'] = 0

for i in range(len(number_of_notes_each_string)):
    if number_of_notes_each_string[i] > 0:
        vibrato_hp_options_perc[i] = (vibrato_hp_options[i] * 100 / number_of_notes_each_string[i]).astype(int)
        vibrato_options_perc[i] = (vibrato_options[i] * 100 / number_of_notes_each_string[i]).astype(int)
    else:
        vibrato_hp_options_perc[i] = 0
        vibrato_options_perc[i] = 0
        
print("Vibrato hp options for each string (percentage): ", vibrato_hp_options_perc)
print("Vibrato options for each string (percentage): ", vibrato_options_perc)
    
general_stats_vibrato = np.array([5,5,6,4,3,2])

if custom_parameters_vibrato:
    if vibrato_stats_deviation == "Y":
            general_stats_vibrato = ((general_stats_vibrato * vibrato_stats_deviation_perc / 100) + general_stats_vibrato).astype(int)

print("Number of notes for each string, 1st to 6th:\n", number_of_notes_each_string)
print("Number of high probability vibrato notes for each string, 1st to 6th: \n", vibrato_hp_options)
print("Number of possible vibrato notes for each string, 1st to 6th: \n", vibrato_options)
print("Number of high probability vibrato notes for each string, 1st to 6th (percentage): \n", vibrato_hp_options_perc)
print("Number of possible vibrato notes for each string, 1st to 6th  (percentage): \n", vibrato_options_perc)
print("General stats about the use of vibrato for each string, 1st to 6th  (percentage):\n", general_stats_vibrato)

# Add the vibrato technique

actual_vibrato_each_string = np.array([0,0,0,0,0,0]) # Count vibrato occurences on each string
actual_vibrato_each_string_perc = np.array([0,0,0,0,0,0]) # Count vibrato occurences percentage on each string

for i in df.index:
    if df.at[i, 'vibrato_prob'] == 2:
        if always_vibrato_on_longer_notes == "Y" or vibrato_hp_options_perc[df.at[i, 'string']-1] <= general_stats_vibrato[df.at[i, 'string']-1]:
            df.at[i, 'vibrato'] = True
            actual_vibrato_each_string[df.at[i, 'string']-1] += 1
        else:
            if vibrato_hp_options_perc[df.at[i, 'string']-1] != 0:
                probability = general_stats_vibrato[df.at[i, 'string']-1] / vibrato_hp_options_perc[df.at[i, 'string']-1] * 100
                if randint(0,100) < probability:
                    df.at[i, 'vibrato'] = True
                    actual_vibrato_each_string[df.at[i, 'string']-1] += 1                   
                    
print("Vibratos for each string: ", actual_vibrato_each_string)
for i in range(len(number_of_notes_each_string)):
    if number_of_notes_each_string[i] > 0:
        actual_vibrato_each_string_perc[i] = (actual_vibrato_each_string[i]/number_of_notes_each_string[i]*100).astype(int)
    else:
        actual_vibrato_each_string_perc[i] = 0                    
                    
for i in df.index:
    if  df.at[i, 'vibrato_prob'] == 1:
        if always_vibrato_on_compatible_notes == "Y" or (vibrato_hp_options_perc[df.at[i, 'string']-1] + vibrato_options_perc[df.at[i, 'string']-1]) <= general_stats_vibrato[df.at[i, 'string']-1]:
            df.at[i, 'vibrato'] = True
            actual_vibrato_each_string[df.at[i, 'string']-1] += 1
        else:
            if vibrato_options_perc[df.at[i, 'string']-1] != 0:
                probability = (general_stats_vibrato[df.at[i, 'string']-1] - actual_vibrato_each_string_perc[df.at[i, 'string']-1]) / vibrato_options_perc[df.at[i, 'string']-1] * 100
                if randint(0,100) < probability:
                    df.at[i, 'vibrato'] = True
                    actual_vibrato_each_string[df.at[i, 'string']-1] += 1
        
print("Vibratos for each string (percentage): ",actual_vibrato_each_string_perc)
print("##################################")

for i in df.index:
    if df.at[i, 'vibrato'] == 0:
        df.at[i, 'vibrato'] = False

print(df.head(60))
df.to_csv('out_bending_vibrato.csv')

## Hammer On/Pull Off

In [ ]:
custom_parameters_legato = input("Do you want to customize the expressive model parameters regarding hammer-on and pull-off? Type Y or N")

if custom_parameters_legato != "N":
    custom_parameters_legato = True
    always_legato_on_compatible_notes = input("Do you want to always apply hammer-on on pull-off to compatible notes? Type Y or N")
    if always_legato_on_compatible_notes == "N":
        legato_stats_deviation =  input("Do you want to modify the target stats for the application of hammer-on and pull-off? Type Y or N")
        if legato_stats_deviation == "Y":
            legato_stats_deviation_perc = int(input("Type the deviation desired, in percentage"))
        else:
            legato_stats_deviation_perc = 0
else:
    custom_parameters_legato = False

if not custom_parameters_legato:
    always_legato_on_compatible_notes = "N"
    legato_stats_deviation_perc = 0
    
hammer_on_frequency_dataset = np.array([3, 3, 2, 3, 3, 7]) # Probability of having a hammer-on for each string, from the first to the sixth
pull_off_frequency_dataset = np.array([8, 5, 4, 3, 5, 1]) # Probability of having a hammer-on for each string, from the first to the sixth

hammer_on_options = np.array([0,0,0,0,0,0]) # Options to insert hammer-on, for each string, first to sixth
pull_off_options = np.array([0,0,0,0,0,0]) # Options to insert pull-off, for each string, first to sixth
hammer_on_options_perc = np.array([0,0,0,0,0,0]) # Options percentage to insert hammer-on, for each string, first to sixth
pull_off_options_perc = np.array([0,0,0,0,0,0]) # Options percentage to insert pull-off, for each string, first to sixth

def legato_test():
    if df.at[i, 'string'] == df.at[i+1, 'string'] and df.at[i, 'note_numb'] != df.at[i+1, 'note_numb'] and df.at[i, 'bending'] == 0  and df.at[i+1, 'bending'] == 0:
        if df.at[i, 'note_numb'] > df.at[i+1, 'note_numb']:
            df.at[i, 'po_prob'] = 1
            pull_off_options[df.at[i, 'string']-1] += 1
        elif df.at[i, 'note_numb'] < df.at[i+1, 'note_numb']:
            df.at[i, 'ho_prob'] = 1
            hammer_on_options[df.at[i, 'string']-1] += 1

for i in range(len(df.index)-1):            
    legato_test()

hammer_on_frequency_dataset_customized = ((hammer_on_frequency_dataset * legato_stats_deviation_perc / 100) + hammer_on_frequency_dataset).astype(int)
pull_off_frequency_dataset_customized = ((pull_off_frequency_dataset * legato_stats_deviation_perc / 100) + pull_off_frequency_dataset).astype(int)
hammer_on_target = (number_of_notes_each_string * hammer_on_frequency_dataset_customized / 100).astype(int)
pull_off_target = (number_of_notes_each_string * pull_off_frequency_dataset_customized / 100).astype(int)

for i in range(len(number_of_notes_each_string)):
    if number_of_notes_each_string[i] > 0:
        hammer_on_options_perc[i] = (hammer_on_options[i] * 100 / number_of_notes_each_string[i]).astype(int)            
        pull_off_options_perc[i] = (pull_off_options[i] * 100 / number_of_notes_each_string[i]).astype(int)
    else:
        hammer_on_options_perc[i] = 0
        pull_off_options_perc[i] = 0

print("Number of notes for each string, 1st to 6th:\n", number_of_notes_each_string)
print("Number of possible hammer-on for each string, 1st to 6th: \n", hammer_on_options)
print("Number of possible pull-off for each string, 1st to 6th: \n", pull_off_options)
print("Number of possible hammer-on for each string, 1st to 6th (percentage): \n", hammer_on_options_perc)
print("Number of possible pull-off for each string, 1st to 6th(percentage): \n", pull_off_options_perc)
print("Target number of hammer-on for each string, 1st to 6th: \n", hammer_on_target)
print("Target number of pull-off for each string, 1st to 6th: \n", pull_off_target)
print("Target percentage of hammer-on for each string, 1st to 6th: \n", hammer_on_frequency_dataset_customized)
print("Target percentage of pull-off for each string, 1st to 6th: \n", pull_off_frequency_dataset_customized)

In [ ]:
actual_number_hammer_on = np.array([0,0,0,0,0,0]) # Actual number of hammer-on, for each string, first to sixth
actual_number_pull_off = np.array([0,0,0,0,0,0]) # Actual number of pull-off, for each string, first to sixth
    
def hammer_on_pull_off():
    if df.at[i, 'string'] == df.at[i+1, 'string'] and df.at[i, 'note_numb'] != df.at[i+1, 'note_numb'] and df.at[i, 'bending'] == 0  and df.at[i+1, 'bending'] == 0:
        if df.at[i, 'note_numb'] > df.at[i+1, 'note_numb']:
            if always_legato_on_compatible_notes == "Y" or pull_off_options_perc[df.at[i, 'string']-1] <= pull_off_frequency_dataset_customized[df.at[i, 'string']-1]:
                df.at[i+1, 'pull_off'] = True
                actual_number_pull_off[df.at[i, 'string']-1] += 1
            elif pull_off_options_perc[df.at[i, 'string']-1] != 0:
                probability = pull_off_frequency_dataset_customized[df.at[i, 'string']-1] / pull_off_options_perc[df.at[i, 'string']-1] * 100
                if randint(0,100) < probability:
                    df.at[i+1, 'pull_off'] = True
                    actual_number_pull_off[df.at[i, 'string']-1] += 1
        elif df.at[i, 'note_numb'] < df.at[i+1, 'note_numb']:
            if always_legato_on_compatible_notes == "Y" or hammer_on_options_perc[df.at[i, 'string']-1] <= hammer_on_frequency_dataset_customized[df.at[i, 'string']-1]:
                df.at[i+1, 'hammer_on'] = True 
                actual_number_hammer_on[df.at[i, 'string']-1] += 1
            elif hammer_on_options_perc[df.at[i, 'string']-1] != 0:
                probability = hammer_on_frequency_dataset_customized[df.at[i, 'string']-1] / hammer_on_options_perc[df.at[i, 'string']-1] * 100
                if randint(0,100) < probability:
                    df.at[i+1, 'hammer_on'] = True
                    actual_number_hammer_on[df.at[i, 'string']-1] += 1
    if df.at[i, 'hammer_on'] == 0:
        df.at[i, 'hammer_on'] = False
    if df.at[i, 'pull_off'] == 0:
        df.at[i, 'pull_off'] = False
        
for i in range(len(df.index)-1):            
    hammer_on_pull_off()
    
df.at[0, 'hammer_on'] = False
df.at[0, 'pull_off'] = False

print("Target number of hammer-on for each string, 1st to 6th: \n", hammer_on_target)
print("Actual number of HO:\n", actual_number_hammer_on)        
print("Target number of pull-off for each string, 1st to 6th: \n", pull_off_target)
print("Actual number of PO:\n", actual_number_pull_off)        
            
print(df[['string', 'fret', 'finger', 'bending', 'vibrato', 'hammer_on', 'pull_off']].head(60))

## Slide legato

In [ ]:
custom_parameters_slide_legato = input("Do you want to customize the expressive model parameters regarding slide legato? Type Y or N")

if custom_parameters_slide_legato != "N":
    custom_parameters_slide_legato = True
    always_slide_legato_on_compatible_notes = input("Do you want to always apply slide-legato to compatible notes? Type Y or N")

for i in range(len(df.index)-1):  
    if custom_parameters_slide_legato:
        if df.at[i, 'string'] == df.at[i+1, 'string'] and df.at[i, 'note_numb'] != df.at[i+1, 'note_numb'] and df.at[i, 'bending'] == 0 and df.at[i+1, 'bending'] == 0 and df.at[i, 'finger'] == df.at[i+1, 'finger']:
            df.at[i, 'slide_legato'] = True
        else:
            df.at[i, 'slide_legato'] = False                
    else:
        df.at[i, 'slide_legato'] = False
        
df.to_csv('dataframe_with_expressiveness.csv')